<a href="https://colab.research.google.com/github/lucas-porto/Data_science_bootcamp/blob/master/Lucas_Porto_Modulo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---
## <font color = #600080>  **Resumo:**
---



Esta análise visa a entender qual a importância a nível Brasil da COVID, como foram os níveis de mortes até agosto de 2020, se o percentual de mortes teve impacto na quantidade de mortes totais e comparada as doenças respiratórias.

## <font color = #8600b3> Introdução:

Começaremos importando as bases iniciais deste estudo, iremos inicialmente avaliar as bases dos dados e verificar os tratamentos necessários para que possamos iniciar o levantamento de hipóteses.

Segundo ponto da análise será verificar a evolução dos óbitos do Brasil, sem contar as taxas de natalidade e nenhum outro fator.

Terceiro ponto será a verificação das doenças respiratórias e como ela evoluiu segundo os dados que o Brasil.IO conseguiu recuperar do Portal de Transparência dos Cartórios. O mesmo portal qual foi origem dos dados do Kaggle.

## <font color = #8600b3>Análise dos dados:


Neste trabalho iremos utilizar 3 bases:

1. Dados de óbitos no Brasil do <a href="https://www.kaggle.com/amandalk/registered-deaths-brazil">Kaggle</a>, a origem dos dados é do <a href="https://transparencia.registrocivil.org.br/registros">Portal de Transparência</a>. Nele temos os registros de todos os estados por meses de 1970 a agosto de 2020.


2. Dados de Covid do Brasil que a a origem é a <a href="https://brasil.io/dataset/covid19/caso_full/">Brasil.io</a>, nele temos aberturas por municípios e estados e são a consolidação dos dados que Secretarias Estaduais de Saúde (SES) encaminham ao Governo Federal.


3. Dados de óbitos de 2019 e 2020 sobre as mortes relacionadas a problemas respiratórios, a origem dos dados é do portal de Transparência dos Cartórios, porém consolidada pelo <a href="https://brasil.io/dataset/covid19/obito_cartorio/">Brasil.io</a>.

### <font color = #8600b3> Carregamento e tratamento dos dados iniciais

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 # Importando bibliotecas
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import MonthEnd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


In [3]:
# Importando as bases
fonte_obitos = 'https://raw.githubusercontent.com/alura-cursos/agendamento-hospitalar/main/dados/obitos-registro-civil.csv' #Obitos do Kaggle
fonte_covid = '/content/drive/My Drive/Estudos/Data Science/Alura/Bootcamp Data Science/Módulo 1/dados/caso_full.csv.gz' #Casos Full - Brasil IO
fonte_cartorio = '/content/drive/My Drive/Estudos/Data Science/Alura/Bootcamp Data Science/Módulo 1/dados/obito_cartorio.csv.gz' #Casos Obitos - Brasil IO

In [4]:
#carregando os dados de óbitos para analisarmos a base
dados_obitos = pd.read_csv(fonte_obitos)
dados_covid = pd.read_csv(fonte_covid, compression = 'gzip')
dados_cartorio = pd.read_csv(fonte_cartorio, compression = 'gzip')

#criada cópia de segurança para qualquer necessidade futura, apesar de alocar espaço em memória.
dados_obitos_copia = dados_obitos.copy()
dados_covid_copia = dados_covid.copy()
dados_cartorio_copia = dados_cartorio.copy()


#### <font color = #8600b3> Verificando os dados de óbitos

In [5]:
#primeiros dados
dados_obitos.head(2)

,year,month,state,deaths
0,1970,1,AM,1
1,1970,1,BA,6


In [6]:
#últimos dados - temos até agosto de 2020
dados_obitos.tail(2)

,year,month,state,deaths
15028,2020,8,SE,1452
15029,2020,8,TO,741


In [7]:
# Aqui temos um panorama de como estão nossos dados, podemos analisar que não temos nenhum dado faltante
# Mas os dados estão quebrados por mês e ano
print(f'A base de óbitos possui um total de dados {dados_obitos.isna().sum().sum()} faltantes.\n')
dados_obitos.info()

A base de óbitos possui um total de dados 0 faltantes.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15030 entries, 0 to 15029
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    15030 non-null  int64 
 1   month   15030 non-null  int64 
 2   state   15030 non-null  object
 3   deaths  15030 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 469.8+ KB


In [8]:
print(f'A base de Óbitos apresenta {dados_obitos.shape[0]} registros e {dados_obitos.shape[1]} colunas.')

A base de Óbitos apresenta 15030 registros e 4 colunas.


##### <font color = #8600b3> Tratando os dados da base de óbitos

In [9]:
dados_obitos.dtypes

year       int64
month      int64
state     object
deaths     int64
dtype: object

In [10]:
#alterando nomes das colunas para português
dados_obitos.rename(columns={'year':'ano', 'month':'mês', 'state': 'estado', 'deaths':'mortes'}, inplace=True)

#criando uma informação de dia
dados_obitos['dia'] = 31 
#realizando a transformação para data
dados_obitos['data'] = dados_obitos['ano'].astype(str) + dados_obitos['mês'].astype(str).str.zfill(2) 
dados_obitos['data'] = pd.to_datetime(dados_obitos['data'], format='%Y%m')  + MonthEnd(0)


In [11]:
dados_obitos.head(2)

,ano,mês,estado,mortes,dia,data
0,1970,1,AM,1,31,1970-01-31
1,1970,1,BA,6,31,1970-01-31


In [12]:
#Como estamos trabalhando com dados mais atuais, iremos analisar apenas os últimos 20 anos e verificar se temos algum insight.
dados_obitos_a_partir_2000 = dados_obitos[['data','estado','mortes','ano']][(dados_obitos['ano'] >= 2000)]
# Agora temos um dataframe mais fácil de utilizar
dados_obitos_a_partir_2000.head(2)

,data,estado,mortes,ano
8360,2000-01-31,AC,57,2000
8361,2000-01-31,AL,176,2000


#### <font color = #8600b3> Verificando os dados de COVID

In [13]:
#primeiros dados
dados_covid.head(2)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,São Paulo,3550308.0,2020-02-25,9,12325232.0,12252023.0,False,False,1,0.00811,2020-02-25,0.0,0,1,city,SP,1,0
1,NaN,35.0,2020-02-25,9,46289333.0,45919049.0,False,False,1,0.00216,2020-02-25,0.0,0,1,state,SP,1,0


In [14]:
#últimos dados - temos até agosto de 2020
dados_covid.tail(2)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1016013,NaN,35.0,2020-11-07,45,46289333.0,45919049.0,False,True,1125936,2432.38761,2020-11-05,0.0353,39717,257,state,SP,0,0
1016014,NaN,17.0,2020-11-07,45,1590248.0,1572866.0,False,True,76739,4825.59953,2020-11-06,0.0145,1112,235,state,TO,0,0


In [15]:
print(f'A base de Óbitos apresenta {dados_covid.shape[0]} registros e {dados_covid.shape[1]} colunas.')

A base de Óbitos apresenta 1016015 registros e 18 colunas.


In [16]:
# Aqui temos um panorama de como estão nossos dados, podemos analisar quantos estão vazios
#last_available_confirmed_per_100k_inhabitants, estimated_population_2019, estimated_population, city_ibge_code e city = Possuem dados faltantes
print('A base de Covid possui a quantidade de valores nulos nas seguintes colunas:')
dados_covid.isna().sum()

A base de Covid possui a quantidade de valores nulos nas seguintes colunas:


city                                              6484
city_ibge_code                                    4051
date                                                 0
epidemiological_week                                 0
estimated_population                              4051
estimated_population_2019                         4051
is_last                                              0
is_repeated                                          0
last_available_confirmed                             0
last_available_confirmed_per_100k_inhabitants    18006
last_available_date                                  0
last_available_death_rate                            0
last_available_deaths                                0
order_for_place                                      0
place_type                                           0
state                                                0
new_confirmed                                        0
new_deaths                                           0
dtype: int

##### <font color = #8600b3>Tratando os dados da base de Covid

Inicialmente iremos avaliar o maior número de vazios que é a coluna: last_available_confirmed_per_100k_inhabitants

Posteriormente iremos verificar se temos alguma correlação entre: *estimated_population* e *city_ibge_code*

Por último veremos o caso da coluna: *city*

Conforme documentação dos dados, irei eliminar a coluna: *estimated_population_2019*

In [17]:
#ordenando e tirando já uma coluna
dados_covid = dados_covid[['state','city', 'place_type', 'city_ibge_code', 'date', 'epidemiological_week',
            'is_last', 'is_repeated', 'new_confirmed', 'new_deaths',
            'last_available_confirmed', 'last_available_confirmed_per_100k_inhabitants',
            'estimated_population','last_available_date', 'last_available_death_rate', 'last_available_deaths',
            'order_for_place']]

In [18]:
dados_covid.dtypes

state                                             object
city                                              object
place_type                                        object
city_ibge_code                                   float64
date                                              object
epidemiological_week                               int64
is_last                                             bool
is_repeated                                         bool
new_confirmed                                      int64
new_deaths                                         int64
last_available_confirmed                           int64
last_available_confirmed_per_100k_inhabitants    float64
estimated_population                             float64
last_available_date                               object
last_available_death_rate                        float64
last_available_deaths                              int64
order_for_place                                    int64
dtype: object

Vamos transformar a data em tipo datetime

In [19]:
dados_covid['date'] = pd.to_datetime(dados_covid['date'], yearfirst=True)

In [20]:
dados_covid.dtypes

state                                                    object
city                                                     object
place_type                                               object
city_ibge_code                                          float64
date                                             datetime64[ns]
epidemiological_week                                      int64
is_last                                                    bool
is_repeated                                                bool
new_confirmed                                             int64
new_deaths                                                int64
last_available_confirmed                                  int64
last_available_confirmed_per_100k_inhabitants           float64
estimated_population                                    float64
last_available_date                                      object
last_available_death_rate                               float64
last_available_deaths                   

In [21]:
print('Verificando quantos dados para o tipo cidade temos de valores nulos:', dados_covid[(dados_covid['place_type'] == 'city')]['city'].isna().sum())
print('Verificando quantos dados para o tipo estado temos campos nulos para cidade:',dados_covid[(dados_covid['place_type'] == 'state')]['city'].isna().sum()) 

Verificando quantos dados para o tipo cidade temos de valores nulos: 0
Verificando quantos dados para o tipo estado temos campos nulos para cidade: 6484


In [22]:
print('Analisando as cidades que contém o campo cidade preencido como Importados/Indefinidos:\n')
dados_covid[(dados_covid['place_type'] == 'city') & (dados_covid['city'] == 'Importados/Indefinidos')]


Analisando as cidades que contém o campo cidade preencido como Importados/Indefinidos:



,state,city,place_type,city_ibge_code,date,epidemiological_week,is_last,is_repeated,new_confirmed,new_deaths,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,estimated_population,last_available_date,last_available_death_rate,last_available_deaths,order_for_place
193,PE,Importados/Indefinidos,city,NaN,2020-03-14,11,False,False,1,0,1,NaN,NaN,2020-03-14,0.0000,0,1
241,PE,Importados/Indefinidos,city,NaN,2020-03-15,12,False,False,0,0,1,NaN,NaN,2020-03-15,0.0000,0,2
295,PE,Importados/Indefinidos,city,NaN,2020-03-16,12,False,True,0,0,1,NaN,NaN,2020-03-15,0.0000,0,3
361,PE,Importados/Indefinidos,city,NaN,2020-03-17,12,False,False,1,0,2,NaN,NaN,2020-03-17,0.0000,0,4
438,PE,Importados/Indefinidos,city,NaN,2020-03-18,12,False,False,1,0,3,NaN,NaN,2020-03-18,0.0000,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014286,RO,Importados/Indefinidos,city,NaN,2020-11-07,45,False,True,0,0,0,NaN,NaN,2020-11-03,0.0000,0,198
1014325,RR,Importados/Indefinidos,city,NaN,2020-11-07,45,False,True,0,0,1493,NaN,NaN,2020-11-04,0.0033,5,192
1014537,RS,Importados/Indefinidos,city,NaN,2020-11-07,45,False,True,0,0,0,NaN,NaN,2020-11-05,0.0000,0,102
1014942,SC,Importados/Indefinidos,city,NaN,2020-11-07,45,False,True,0,0,7016,NaN,NaN,2020-11-04,0.0000,0,228


In [23]:
#Vou separar as mortes por estado, selecionando apenas as que tem o place_type == state
dados_covid_estados = dados_covid[dados_covid['place_type'] == 'state']
dados_covid_estados.set_index('state', inplace=True)
dados_covid_estados.tail()

,city,place_type,city_ibge_code,date,epidemiological_week,is_last,is_repeated,new_confirmed,new_deaths,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,estimated_population,last_available_date,last_available_death_rate,last_available_deaths,order_for_place
state,,,,,,,,,,,,,,,,
RS,NaN,state,43.0,2020-11-07,45,True,False,2403,17,260188,2277.76079,11422973.0,2020-11-07,0.0230,5990,243
SC,NaN,state,42.0,2020-11-07,45,True,False,5008,29,273652,3773.20820,7252502.0,2020-11-07,0.0117,3202,241
SE,NaN,state,28.0,2020-11-07,45,True,False,173,4,85226,3675.40070,2318822.0,2020-11-07,0.0263,2239,239
SP,NaN,state,35.0,2020-11-07,45,False,True,0,0,1125936,2432.38761,46289333.0,2020-11-05,0.0353,39717,257
TO,NaN,state,17.0,2020-11-07,45,False,True,0,0,76739,4825.59953,1590248.0,2020-11-06,0.0145,1112,235


In [24]:
#Podemos ignorar a coluna city quando estamos lidando com estados. 
# Temos também problemas com o last_available_confirmed_per_100k_inhabitants. Caso queiramos algo
#teremos que criar uma calculada
dados_covid_estados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6484 entries, SP to TO
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   city                                           0 non-null      object        
 1   place_type                                     6484 non-null   object        
 2   city_ibge_code                                 6484 non-null   float64       
 3   date                                           6484 non-null   datetime64[ns]
 4   epidemiological_week                           6484 non-null   int64         
 5   is_last                                        6484 non-null   bool          
 6   is_repeated                                    6484 non-null   bool          
 7   new_confirmed                                  6484 non-null   int64         
 8   new_deaths                                     6484 non-null   i

In [25]:
quantidade_de_estados_com_ultimo_report = len(dados_covid_estados[dados_covid_estados['is_last']].index)
print(f'Temos um total de {quantidade_de_estados_com_ultimo_report} com último reporte.')

Temos um total de 27 com último reporte.


In [26]:
print('Os dados tem a última data de atualização com a seguinte distribuição:')
dados_covid_estados[dados_covid_estados['is_last'] == True][['date','last_available_deaths','last_available_death_rate','last_available_confirmed','last_available_confirmed_per_100k_inhabitants']]

Os dados tem a última data de atualização com a seguinte distribuição:


,date,last_available_deaths,last_available_death_rate,last_available_confirmed,last_available_confirmed_per_100k_inhabitants
state,,,,,
AP,2020-11-03,751,0.0142,52832,6130.61676
SP,2020-11-05,39717,0.0353,1125936,2432.38761
MS,2020-11-06,1634,0.0193,84720,3015.59696
RN,2020-11-06,2607,0.0315,82719,2340.55286
RR,2020-11-06,695,0.0118,59065,9357.85456
TO,2020-11-06,1112,0.0145,76739,4825.59953
AC,2020-11-07,699,0.0221,31575,3530.02337
AL,2020-11-07,2266,0.0247,91603,2733.15903
AM,2020-11-07,4622,0.0279,165394,3930.73293


#### <font color = #8600b3> Verificando os dados de Óbitos por doença respiratória

In [27]:
#primeiros dados
dados_cartorio.head(2)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020,new_deaths_indeterminate_2019,new_deaths_respiratory_failure_2019,new_deaths_others_2019,new_deaths_pneumonia_2019,new_deaths_septicemia_2019,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
0,2020-01-01,AC,1,1,0,2,4,3,1,0,0,1,1,7,2,2,0,10,13,NaN,2.0,4.0,3.0,1.0,NaN,NaN,1.0,1.0,7.0,2.0,2.0,NaN,10,13.0
1,2020-01-02,AC,1,1,0,2,10,3,2,0,0,1,1,13,4,4,0,17,23,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,NaN,6.0,2.0,2.0,NaN,7,10.0


In [28]:
#últimos dados
dados_cartorio.tail(2)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020,new_deaths_indeterminate_2019,new_deaths_respiratory_failure_2019,new_deaths_others_2019,new_deaths_pneumonia_2019,new_deaths_septicemia_2019,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
9880,2020-12-30,TO,1,53,48,451,3134,893,514,8,790,41,380,2710,671,479,24,5048,5095,NaN,3.0,10.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN
9881,2020-12-31,TO,1,53,48,452,3144,894,516,8,790,41,380,2710,671,479,24,5062,5095,NaN,1.0,10.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN


In [29]:
print(f'A base de Óbitos apresenta {dados_cartorio.shape[0]} registros de imóveis e {dados_cartorio.shape[1]} colunas.')

A base de Óbitos apresenta 9882 registros de imóveis e 34 colunas.


In [30]:
# Aqui temos um panorama de como estão nossos dados, podemos analisar quantos estão vazios
#
print('A base de Óbitos por doença respiratória possui a quantidade de valores nulos nas seguintes colunas:')
dados_cartorio.isna().sum()

A base de Óbitos por doença respiratória possui a quantidade de valores nulos nas seguintes colunas:


date                                      0
state                                     0
epidemiological_week_2019                 0
epidemiological_week_2020                 0
deaths_indeterminate_2019                 0
deaths_respiratory_failure_2019           0
deaths_others_2019                        0
deaths_pneumonia_2019                     0
deaths_septicemia_2019                    0
deaths_sars_2019                          0
deaths_covid19                            0
deaths_indeterminate_2020                 0
deaths_respiratory_failure_2020           0
deaths_others_2020                        0
deaths_pneumonia_2020                     0
deaths_septicemia_2020                    0
deaths_sars_2020                          0
deaths_total_2019                         0
deaths_total_2020                         0
new_deaths_indeterminate_2019          6606
new_deaths_respiratory_failure_2019    1063
new_deaths_others_2019                   52
new_deaths_pneumonia_2019       

Avaliando mais a fundo os dados, vemos que está ordenado por dia do ano e por estado em ordem alfabética, os casos nulos o que podemos avaliar pelas primeiras linhas são todos referentes aos new_deaths, caso não tenha um novo caso ao invés de 0 ele carrega o NaN.


Ou seja podemos ignorar todos os new_deaths e se quisermos a evolução, ou seja os casos diários, só fazer um .diff() para a coluna desejada.

In [31]:
#Avaliando mais a fundo os dados, vemos que está ordenado por dia do ano e por estado em ordem alfabética
dados_cartorio.head(367)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020,new_deaths_indeterminate_2019,new_deaths_respiratory_failure_2019,new_deaths_others_2019,new_deaths_pneumonia_2019,new_deaths_septicemia_2019,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
0,2020-01-01,AC,1,1,0,2,4,3,1,0,0,1,1,7,2,2,0,10,13,NaN,2.0,4.0,3.0,1.0,NaN,NaN,1.0,1.0,7.0,2.0,2.0,NaN,10,13.0
1,2020-01-02,AC,1,1,0,2,10,3,2,0,0,1,1,13,4,4,0,17,23,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,NaN,6.0,2.0,2.0,NaN,7,10.0
2,2020-01-03,AC,1,1,0,2,16,3,4,0,0,1,1,21,7,6,0,25,36,NaN,NaN,6.0,NaN,2.0,NaN,NaN,NaN,NaN,8.0,3.0,2.0,NaN,8,13.0
3,2020-01-04,AC,1,1,0,2,18,6,5,0,0,1,2,25,7,7,0,31,42,NaN,NaN,2.0,3.0,1.0,NaN,NaN,NaN,1.0,4.0,NaN,1.0,NaN,6,6.0
4,2020-01-05,AC,1,2,0,2,24,7,5,0,0,1,3,29,11,7,0,38,51,NaN,NaN,6.0,1.0,NaN,NaN,NaN,NaN,1.0,4.0,4.0,NaN,NaN,7,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2020-12-28,AC,52,53,35,213,1939,855,404,5,709,34,213,1848,558,348,24,3451,3734,NaN,NaN,6.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
363,2020-12-29,AC,1,53,35,213,1945,856,404,5,709,34,213,1848,558,348,24,3458,3734,NaN,NaN,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN
364,2020-12-30,AC,1,53,35,214,1957,859,405,5,709,34,213,1848,558,348,24,3475,3734,NaN,1.0,12.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN
365,2020-12-31,AC,1,53,35,214,1962,866,406,5,709,34,213,1848,558,348,24,3488,3734,NaN,NaN,5.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN


##### <font color = #8600b3> Tratando os dados da base de óbitos por doença respiratória

In [32]:
dados_cartorio.dtypes

date                                    object
state                                   object
epidemiological_week_2019                int64
epidemiological_week_2020                int64
deaths_indeterminate_2019                int64
deaths_respiratory_failure_2019          int64
deaths_others_2019                       int64
deaths_pneumonia_2019                    int64
deaths_septicemia_2019                   int64
deaths_sars_2019                         int64
deaths_covid19                           int64
deaths_indeterminate_2020                int64
deaths_respiratory_failure_2020          int64
deaths_others_2020                       int64
deaths_pneumonia_2020                    int64
deaths_septicemia_2020                   int64
deaths_sars_2020                         int64
deaths_total_2019                        int64
deaths_total_2020                        int64
new_deaths_indeterminate_2019          float64
new_deaths_respiratory_failure_2019    float64
new_deaths_ot

In [33]:
dados_cartorio['date'] = pd.to_datetime(dados_cartorio['date'], yearfirst=True)

In [34]:
#Separando até a data de agosto que será avaliada mais a frente.
#Retirando as colunas de new_deaths pois iremos calcular as que precisaremos
dados_cartorio = dados_cartorio[dados_cartorio['date'] <= '2020-08-31'].loc[:,~dados_cartorio.columns.str.startswith('new_deaths')]

In [35]:
dados_cartorio.head(5)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020
0,2020-01-01,AC,1,1,0,2,4,3,1,0,0,1,1,7,2,2,0,10,13
1,2020-01-02,AC,1,1,0,2,10,3,2,0,0,1,1,13,4,4,0,17,23
2,2020-01-03,AC,1,1,0,2,16,3,4,0,0,1,1,21,7,6,0,25,36
3,2020-01-04,AC,1,1,0,2,18,6,5,0,0,1,2,25,7,7,0,31,42
4,2020-01-05,AC,1,2,0,2,24,7,5,0,0,1,3,29,11,7,0,38,51


In [36]:
#para criar as colunas de new_deaths para cada coluna irei criar uma função para deixar mais rápido a tratativa
#Palmas ou o Tocantis inteiro para a equipe do ScubaTeam que resolveu essa belezura
def criar_colunas(dataframe, parte_nome_coluna):
  columns = dataframe.columns[dataframe.columns.str.startswith(parte_nome_coluna)]
  i=0
  for coluna in columns:
    nova_coluna = 'new_'+columns[i]
    dataframe[nova_coluna] = dataframe.groupby('state')[coluna].diff().fillna(dataframe[coluna])
    i+=1
  dataframe.fillna(0, inplace=True)

In [37]:
criar_colunas(dados_cartorio, 'deaths')
dados_cartorio.head(5)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020,new_deaths_indeterminate_2019,new_deaths_respiratory_failure_2019,new_deaths_others_2019,new_deaths_pneumonia_2019,new_deaths_septicemia_2019,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
0,2020-01-01,AC,1,1,0,2,4,3,1,0,0,1,1,7,2,2,0,10,13,0.0,2.0,4.0,3.0,1.0,0.0,0.0,1.0,1.0,7.0,2.0,2.0,0.0,10.0,13.0
1,2020-01-02,AC,1,1,0,2,10,3,2,0,0,1,1,13,4,4,0,17,23,0.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,2.0,2.0,0.0,7.0,10.0
2,2020-01-03,AC,1,1,0,2,16,3,4,0,0,1,1,21,7,6,0,25,36,0.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,3.0,2.0,0.0,8.0,13.0
3,2020-01-04,AC,1,1,0,2,18,6,5,0,0,1,2,25,7,7,0,31,42,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,6.0,6.0
4,2020-01-05,AC,1,2,0,2,24,7,5,0,0,1,3,29,11,7,0,38,51,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,7.0,9.0


### <font color = #8600b3> Iniciando as Hipóteses

Para tornar mais intuitivo irei elencar inicalmente algumas hipóteses a serem analisadas:

- Como era a série histórica de mortes no país? Será que a COVID foi uma influência tão grande nos óbitos?


- Como foi a evolução das mortes de doenças respiratórias de 2019 para 2020?


#### <font color = #8600b3> Como era a série histórica de mortes no país? Será que a COVID foi uma influência tão grande nos óbitos?

Como já havia separado inicalmente iremos analisar a partir de 2000, a fim de restringir um pouco a análise e os gráficos não ficarem com tempo muito alongado, já que no fim a idéia é verificarmos o que está acontecendo em 2020.

Lembrando que nossa base de óbitos está restrita até Agosto de 2020, iremos a fim de verificarmos por gráficos restringir a base de Covid ao mesmo tempo. 

In [38]:
#Selecionando os óbitos de 2020
dados_obitos_2020 = dados_obitos_a_partir_2000[dados_obitos_a_partir_2000['ano'] >= 2020]
dados_obitos_2020_agrupados = dados_obitos_2020.groupby('data').sum()

#Selecionando os dados de covid até 31-08-2020
dados_covid_estados_ate_agosto_2020 = dados_covid_estados[dados_covid_estados['date'] <= '2020-08-31']
#selecionando apenas as colunas necessárias
dados_covid_estados_ate_agosto_2020_agregado = dados_covid_estados_ate_agosto_2020.groupby('date').sum()[['new_deaths','new_confirmed']]

In [41]:
dados_covid_estados_ate_agosto_2020.groupby('date').sum()[['new_confirmed']]

,new_confirmed
date,
2020-02-25,1
2020-02-26,0
2020-02-27,0
2020-02-28,1
2020-02-29,0
...,...
2020-08-27,42260
2020-08-28,44088
2020-08-29,37291


In [84]:
#@title Distribuição mensal de óbitos totais e óbitos por COVID


fig = go.Figure()

fig.add_trace(go.Scatter( x = dados_obitos_2020_agrupados.index, y = dados_obitos_2020_agrupados['mortes'], 
                         mode='lines',
                         name='Óbitos Totais',  
                         line=dict(color='rgb(49,130,189)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_covid_estados_ate_agosto_2020_agregado.index, y = dados_covid_estados_ate_agosto_2020_agregado['new_deaths'], 
                         mode='lines',
                         name='Óbitos COVID',  
                         line=dict(color='rgb(0, 0, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter(
    x=['2020-02-25', '2020-03-17'],
    y=[140000, 70000],
    text=['Início do acompanhamento',
          'Primeiro registro de Morte por COVID'],
    mode="text",
    showlegend=False,
))

fig.add_shape(type="line",
    x0='2020-02-25', y0=0, x1='2020-02-25', y1=140000,
    line=dict(color="MediumPurple",width=2),
    name='Início do acompanhamento',
)

fig.update_traces(textposition='top right')

fig.add_shape(type="line",
    x0='2020-03-17', y0=0, x1='2020-03-17', y1=120000,
    line=dict(color="RoyalBlue",width=2),
    name='Primeiro registro de Morte por COVID',
)

fig.update_layout(
    title_text = 'Distribuição mensal de óbitos totais e óbitos por COVID',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = 'Meses',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Analisando friamente os números de mortes não temos uma indicação que a COVID é um problema real, já que o maior problema não é a mortalidade e sim a falta de tratamento adequado. 

O sistema de saúde brasileiro entrenta diversos <a href="https://www.ceen.com.br/os-6-maiores-desafios-que-a-saude-publica-do-brasil-vem-enfrentando/">desafios</a>, e a superlotação é a maior preocupação já que a <a href="https://www.scielo.br/scielo.php?pid=S0103-05822020000100101&script=sci_arttext&tlng=pt">transmissibilidade</a> do coronavírus é alta.

Vamos dar uma olhada mais a fundo no COVID e verificar as evolução de contaminação e mortes no país.


In [43]:
#@title Distribuição mensal de óbitos e infecções por COVID

fig = go.Figure()

fig.add_trace(go.Scatter( x = dados_covid_estados_ate_agosto_2020_agregado.index, y = dados_covid_estados_ate_agosto_2020_agregado['new_confirmed'], 
                         mode='lines',
                         name='Novos casos de COVID',  
                         line=dict(color='rgb(0, 0, 77)',width=3)
                         )
    
)
fig.add_trace(go.Scatter( x = dados_covid_estados_ate_agosto_2020_agregado.index, y = dados_covid_estados_ate_agosto_2020_agregado['new_deaths'], 
                         mode='lines',
                         name='Óbitos COVID',  
                         line=dict(color='rgb(73, 73, 208)',width=3)
                         )
    
)

ponto_acompanhamento = max(dados_covid_estados_ate_agosto_2020_agregado['new_confirmed'])*1.1
ponto_morte = max(dados_covid_estados_ate_agosto_2020_agregado['new_confirmed'])*1.0
ponto_pico = max(dados_covid_estados_ate_agosto_2020_agregado['new_confirmed'])

fig.add_trace(go.Scatter(
    x=['2020-02-25', '2020-03-17', '2020-07-22'],
    y=[ponto_acompanhamento, ponto_morte,ponto_pico],
    text=['Início do acompanhamento',
          'Primeiro registro de Morte por COVID',
          'Recorde de novos casos de COVID'],
    mode="text",
    showlegend=False,
))


fig.add_shape(type="line",
    x0='2020-02-25', y0=0, x1='2020-02-25', y1=ponto_acompanhamento,
    line=dict(color='rgb(204, 204, 0)',width=2),
    name='Início do acompanhamento',
)


fig.add_shape(type="line",
    x0='2020-03-17', y0=0, x1='2020-03-17', y1=ponto_morte,
    line=dict(color='rgb(26, 102, 0)',width=2),
    name='Primeiro registro de Morte por COVID',
)

fig.update_traces(textposition='top right')


fig.update_layout(
    title_text = 'Distribuição mensal de óbitos e infecções por COVID',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = 'Meses',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Verificando os gráficos de novos casos, temos duas interpretações, a primeira que até o final de agosto tivemos um aumento significativo de novos casos, com uma leve diminuição no final de agosto levando a quase uma estabilidade.

A segunda análise é que nos finais de semana temos grandes problemas de atualização, tendo alguns estados não atualizando seus dados, dando picos nos dias subsequentes aos finais de semana.

Apesar de não bater exatamente com o que foi reportado pela <a href="https://www.dw.com/pt-br/com-67-mil-novos-casos-de-covid-19-brasil-bate-recorde-diário/a-54271031">Deutsche Welle</a>, temos o mesmo dia de maior registro de infectados no país.

In [44]:
dados_covid_estados_agosto_2020 = dados_covid_estados_ate_agosto_2020[dados_covid_estados_ate_agosto_2020['date'] == '2020-08-31'][['last_available_deaths', 'last_available_confirmed']].sum().to_frame().rename({0:'Quantity'}, axis='columns').T
dados_covid_estados_agosto_2020['death_rate'] = dados_covid_estados_agosto_2020['last_available_deaths'] / dados_covid_estados_agosto_2020['last_available_confirmed']

confirmados = dados_covid_estados_agosto_2020['last_available_confirmed'].values[0]
mortos = dados_covid_estados_agosto_2020['last_available_deaths'].values[0]
percentual = round(dados_covid_estados_agosto_2020['death_rate'].values[0]*100,2)

print(f'Ao final de agosto chegamos ao total de {confirmados} confirmados, onde tivemos um total de {mortos} mortes e um percentual de mortes de {percentual}%.')

Ao final de agosto chegamos ao total de 3914829 confirmados, onde tivemos um total de 121618 mortes e um percentual de mortes de 3.11%.


##### <font color = #600080>  **Conclusão da hipótese:**

Apesar de não ser um percentual muito grande comparado ao total de mortes no até agosto de 2020, vimos que temos um percentual dentro dos dados divulgados de 3,11% de taxa de mortalidade e tivemos um total de 121.618 mortes e 3.914.829 casos confirmados.

Vimos por outras fontes que a maior preocupação é a alta taxa de transmissão do vírus, que leva a superlotação dos hospitais em pouco tempo. Sem sabermos ainda se há uma imunidade após a infecção, caso não tivessem sido adotadas as medidas de contenção e distanciamento, poderíamos ter uma catástrofe no país com todos os hospitais lotados e infectados não sendo devidamente tratados.

Então, ela é muito problemática e não podemos analisar friamente apenas os dados de mortes em comparação as mortes totais do país.

#### <font color = #8600b3>  Como foi a evolução das mortes de doenças respiratórias de 2019 para 2020?

Conforme vimos anteriormente temos que o COVID tem seu problema maior é o <a href="https://www.sanarmed.com/como-uma-pandemia-pode-afetar-o-sistema-de-saude">congestionamento</a> do sistema de saúde podendo levar a mais mortes devido ao não tratamento correto dos infectados.

Assim irei analisar rapidamente para verificar como foi até o final de agosto deste ano, para se manter no mesmo tempo analisado anteriormente, as mortes por doenças respiratórias e como o COVID influenciou ou não influenciou.



In [45]:
#Lembrando como está nosso dataset
dados_cartorio.tail(2)

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,deaths_respiratory_failure_2020,deaths_others_2020,deaths_pneumonia_2020,deaths_septicemia_2020,deaths_sars_2020,deaths_total_2019,deaths_total_2020,new_deaths_indeterminate_2019,new_deaths_respiratory_failure_2019,new_deaths_others_2019,new_deaths_pneumonia_2019,new_deaths_septicemia_2019,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
9758,2020-08-30,TO,35,36,38,299,2086,547,350,3,518,33,300,2064,544,371,17,3323,3847,0.0,4.0,7.0,1.0,3.0,0.0,11.0,1.0,1.0,13.0,1.0,1.0,0.0,15.0,28.0
9759,2020-08-31,TO,35,36,38,300,2101,549,353,3,524,33,301,2074,547,372,17,3344,3868,0.0,1.0,15.0,2.0,3.0,0.0,6.0,0.0,1.0,10.0,3.0,1.0,0.0,21.0,21.0


In [46]:
dados_cartorio_agrupados = dados_cartorio.groupby('date').sum()
#retirei o dia bisexto apenas para ficar mais lean o gráfico.
dados_cartorio_agrupados_sem_29_02 = dados_cartorio_agrupados.loc[ dados_cartorio_agrupados.index != '2020-02-29']

In [47]:
#@title Distribuição mensal de óbitos nos anos de 2019 e 2020

fig = go.Figure()

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'], 
                         mode='lines',
                         name='Mortes 2020',  
                         line=dict(color='rgb(0, 0, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2019'], 
                         mode='lines',
                         name='Mortes 2019',  
                         line=dict(color='rgb(73, 73, 208)',width=3)
                         )
    
)

ponto_acompanhamento = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.1
ponto_morte = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.0

fig.add_trace(go.Scatter(
    x=['2020-02-25', '2020-03-17', '2020-07-22'],
    y=[ponto_acompanhamento, ponto_morte],
    text=['Início do acompanhamento',
          'Primeiro registro de Morte por COVID'],
    mode="text",
    showlegend=False,
))


fig.add_shape(type="line",
    x0='2020-02-25', y0=0, x1='2020-02-25', y1=ponto_acompanhamento,
    line=dict(color='rgb(204, 204, 0)',width=2),
    name='Início do acompanhamento',
)


fig.add_shape(type="line",
    x0='2020-03-17', y0=0, x1='2020-03-17', y1=ponto_morte,
    line=dict(color='rgb(26, 102, 0)',width=2),
    name='Primeiro registro de Morte por COVID',
)

fig.update_traces(textposition='top right')

fig.update_layout(
    title_text = 'Distribuição mensal de óbitos nos anos de 2019 e 2020',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = 'Meses',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Aqui temos um pequeno problema, 2020 é um ano bisexto e acabou gerando aquele dente que fica zerado em 2019. Isso é muito triste, não irei tratar a priori a fim de manter os dados conforme vieram da base e não alterarmos o valor acumulado.

Mas o que importa na análise é que até a primeira semana de Março temos um comportamente muito parecido entre os dois anos que começam a se distanciar por volta da segunda quinzeana de abril.

Algumas discussões podem ser incluídas neste caso, já que o Brasil está muito <a href="https://www.bbc.com/portuguese/brasil-54478219">aquém</a> nos teste, tanto dos infectados ou possíveis infectados. 

Assim, podemos ter muitos mais mortos e a "boca do jacaré" ser muito maior.

**Quesitonamento:** Mas ainda dentro destas mortes, será que o COVID foi o responsável por todo esse crescimento?




In [48]:
#@title Distribuição mensal de óbitos totais e das doenças respiratórias no ano de 2020

fig = go.Figure()

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'], 
                         mode='lines',
                         name='Mortes 2020',  
                         line=dict(color='rgb(0, 0, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2019'], 
                         mode='lines',
                         name='Mortes 2019',  
                         line=dict(color='rgb(73, 73, 208)',width=3)
                         )
    
)




fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_sars_2020'], 
                         mode='lines',
                         name='SARS 2020',  
                         line=dict(color='rgb(128, 0, 128)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_septicemia_2020'], 
                         mode='lines',
                         name='Septicemia 2020',  
                         line=dict(color='rgb(153, 179, 255)',width=3)
                         )
    
)


fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_pneumonia_2020'], 
                         mode='lines',
                         name='Pneumonia 2020',  
                         line=dict(color='rgb(173, 173, 235)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_others_2020'], 
                         mode='lines',
                         name='Outros 2020',  
                         line=dict(color='rgb(179, 179, 230)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_covid19'], 
                         mode='lines',
                         name='COVID19',  
                         line=dict(color='rgb(115, 38, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_indeterminate_2020'], 
                         mode='lines',
                         name='Indeterminados 2020',  
                         line=dict(color='rgb(204, 204, 255)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_respiratory_failure_2020'], 
                         mode='lines',
                         name='Falha Respiratória 2020',  
                         line=dict(color='rgb(179, 204, 230)',width=3)
                         )
    
)



ponto_acompanhamento = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.1
ponto_morte = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.0

fig.add_trace(go.Scatter(
    x=['2020-02-25', '2020-03-17', '2020-07-22'],
    y=[ponto_acompanhamento, ponto_morte],
    text=['Início do acompanhamento',
          'Primeiro registro de Morte por COVID'],
    mode="text",
    showlegend=False,
))


fig.add_shape(type="line",
    x0='2020-02-25', y0=0, x1='2020-02-25', y1=ponto_acompanhamento,
    line=dict(color='rgb(204, 204, 0)',width=2),
    name='Início do acompanhamento',
)


fig.add_shape(type="line",
    x0='2020-03-17', y0=0, x1='2020-03-17', y1=ponto_morte,
    line=dict(color='rgb(26, 102, 0)',width=2),
    name='Primeiro registro de Morte por COVID',
)

fig.update_traces(textposition='top right')

fig.update_layout(
    title_text = 'Distribuição mensal de óbitos totais e das doenças respiratórias no ano de 2020',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = 'Meses',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Nesta análise pode ser confusa a priori, porém como estamos usando uma gráfico interativo, podemos fazer a seleção das doenças que não queremos visualizar apenas clicando na legenda abaixo.

Aqui podemos ver claramente que os casos de mortes por Covid19 tiveram uma evolução muito rápida até metade de maio e depois teve uma leve estabilizada.

Entretando ainda temos muito a aprender sobre as definições de mortes por COVID19, já que temos casos de estados e municípios que não testam os mortos e não temos uma padronização no Brasil para definição dessas mortes, ficando a cargo do responsável informar qual foi o CID.

In [89]:
#@title Distribuição mensal de óbitos totais e das doenças respiratórias no ano de 2020

fig = go.Figure()

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'], 
                         mode='lines',
                         name='Mortes 2020',  
                         line=dict(color='rgb(0, 0, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2019'], 
                         mode='lines',
                         name='Mortes 2019',  
                         line=dict(color='rgb(73, 73, 208)',width=3)
                         )
    
)




fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_sars_2020'], 
                         mode='lines',
                         name='SARS 2020',  
                         line=dict(color='rgb(128, 0, 128)',width=3)
                         )
    
)



fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_covid19'], 
                         mode='lines',
                         name='COVID19',  
                         line=dict(color='rgb(115, 38, 77)',width=3)
                         )
    
)

fig.add_trace(go.Scatter( x = dados_cartorio_agrupados_sem_29_02.index, y = dados_cartorio_agrupados_sem_29_02['new_deaths_total_2019']+dados_cartorio_agrupados_sem_29_02['new_deaths_covid19']\
                         +dados_cartorio_agrupados_sem_29_02['new_deaths_sars_2020'], 
                         mode='lines',
                         name='Mortes 2019 + COVID19 + SARS 2020',  
                         line=dict(color='rgb(230, 46, 0)',width=3, dash='dot')
                         )
    
)




ponto_acompanhamento = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.1
ponto_morte = max(dados_cartorio_agrupados_sem_29_02['new_deaths_total_2020'])*1.0

fig.add_trace(go.Scatter(
    x=['2020-02-25', '2020-03-17', '2020-07-22'],
    y=[ponto_acompanhamento, ponto_morte],
    text=['Início do acompanhamento',
          'Primeiro registro de Morte por COVID'],
    mode="text",
    showlegend=False,
))


fig.add_shape(type="line",
    x0='2020-02-25', y0=0, x1='2020-02-25', y1=ponto_acompanhamento,
    line=dict(color='rgb(204, 204, 0)',width=2),
    name='Início do acompanhamento',
)


fig.add_shape(type="line",
    x0='2020-03-17', y0=0, x1='2020-03-17', y1=ponto_morte,
    line=dict(color='rgb(26, 102, 0)',width=2),
    name='Primeiro registro de Morte por COVID',
)

fig.update_traces(textposition='top right')

fig.update_layout(
    title_text = 'Distribuição mensal de óbitos totais e das doenças respiratórias no ano de 2020',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = 'Meses',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Separamos neste outro gráfico um comparativo das mortes de 2020 e 2019, além de COVID19 e SARS no ano 2020.

A fim didático e experimental, fiz uma simulação realizando a soma das mortes de 2019, com COVID19 e SARS no ano de 2020 e podemos ver que no comparativo, tirando que as mortes não seguem uma métrica linear, o ano de 2020 se comporta muito similar ao ano de 2019 acrescido do avanço das doenças de COVID19 e SARS.

In [80]:
#Criando um dataframe auxiliar para criarmos o comparativo total de mortes.

somatorio = dados_cartorio_agrupados_sem_29_02.sum().to_frame()

nome_mortes = ['Indeterminado', 'Falha do Sistema Respiratório', 'Outras', 'Pneumonia', 'Septicemia', 'SARS', 'COVID19', 'Totais']

somatorio_novos_2019 = [somatorio.loc['new_deaths_indeterminate_2019'][0], somatorio.loc['new_deaths_respiratory_failure_2019'][0],
                        somatorio.loc['new_deaths_others_2019'][0], somatorio.loc['new_deaths_pneumonia_2019'][0],
                        somatorio.loc['new_deaths_septicemia_2019'][0], somatorio.loc['new_deaths_sars_2019'][0],
                        0, somatorio.loc['new_deaths_total_2019'][0]]

somatorio_novos_2020 = [somatorio.loc['new_deaths_indeterminate_2020'][0], somatorio.loc['new_deaths_respiratory_failure_2020'][0],
                        somatorio.loc['new_deaths_others_2020'][0], somatorio.loc['new_deaths_pneumonia_2020'][0],
                        somatorio.loc['new_deaths_septicemia_2020'][0], somatorio.loc['new_deaths_sars_2020'][0],
                        somatorio.loc['new_deaths_covid19'][0], somatorio.loc['new_deaths_total_2020'][0]]

In [88]:
#@title Comparativo de mortes por problemas respiratórios de 2019 a 2020 até o mês do agosto

fig = go.Figure()

fig.add_trace(go.Bar( x = nome_mortes, y = somatorio_novos_2019, 
                         name='Mortes 2019',
                         marker_color = 'rgb(73, 73, 208)'
                         )
)

fig.add_trace(go.Bar( x = nome_mortes, y = somatorio_novos_2020, 
                         name='Mortes 2020',
                         marker_color = 'rgb(0, 0, 77)'
                         )
)


fig.update_layout(
    title_text = 'Comparativo de mortes por problemas respiratórios de 2019 a 2020 até o mês do agosto',
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
        linewidth=2,
        ticks='outside',
        title_text = '',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(41, 41, 41)',
        ),
    ),
    yaxis=dict(
        rangemode = 'tozero',
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(41, 41, 41)',
    ),
    autosize=False,
    width=1500,
    height=900,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    legend=dict(orientation="h", y=-0.15, yanchor="bottom", xanchor="center", x = 0.5),
    plot_bgcolor='white'
)


Por fim, podemos comparar dentro das classificações  que tivemos até agosto de 2020 uma diminuição nos casos registrados de mortes por Pneumonia e Spticemia, porém tivemos um leve aumento nos casos indeterminados, no caso de SARS e no de outras causas.

Como COVID19 é uma doença apenas do ano de 2020 não teremos comparação para esta infecção.

Um ponto interessante de analisar que até agosto as mortes de COVID19 dentro dos dados registrados apresenta uma proporção muito parecida com Pneumonia e Septicemia.

Apenas um leve conceituamento:
1. Sepse ou Septcemia é uma infeção generalizada que era conhecida como infecção no sangue. Ela tem uma taxa muito alta de mortalidade e até hoje é um problema para o Brasil.

*   <a href="https://www.vidaeacao.com.br/taxa-de-mortalidade-por-sepse-no-brasil-e-25-maior/">Taxa de moralidade 25% </a> 
*   <a href="https://drauziovarella.uol.com.br/infectologia/brasil-tem-cerca-de-670-mil-casos-de-sepse-por-ano/">Casos por ano </a> 


2. Pneumonia é uma doença respiratória muito conhecida e até possui um dia de conscientização, 12/11, criado pela OMS. 
*   <a href="https://www.anahp.com.br/noticias/noticias-do-mercado/">Pneumonia atinge mais idosos</a>
*   <a href="http://bvsms.saude.gov.br/ultimas-noticias/3085-12-11-dia-mundial-da-pneumonia">Dia Mundial da Pneumonia</a> 



##### <font color = #600080> **Conclusão da hipótese:**

Não tenho base científica para afirmar que foram as doenças de SARS e COVID19 que fizeram com que a quantidade de registro de óbtidos tivessem a grande elevação de casos no comparativo do ano de 2020 com 2019. Porém os como podemos analisar tivemos uma pequena diminuição nos casos de doenças que tem alta taxa de mortalidade anual, como Pneumonia e Sepse ao mesmo tempo que o COVID19 e SARS apareceram nos registros de óbitos.

O que podemos tirar daqui é que com a nova aparição podem ter tido casos que foram registrados para uma doença ou outra e elas podem ter ocorrido em concomitância e apenas um registro efetuado.


---
## <font color = #8600b3> Considerações finais:
---

Ao analisar a COVID no comparativo 2020 e 2019, dentro das doenças respiratórias ou infecciosas podemos ver que ela teve a mortalidade parecida como doenças que aflingem o país como Pneumonia e Sepse e isso nos traz uma preocupação já que convivemos com essas doenças a anos e não temos uma vacina que as cure definitivamente como a Poliomelite por exemplo.

Quando analisado o percentual de mortes de COVID com o total temos a percepção de não ser um grande problema de saúde pública, porém ao analisarmos a quantidade de infectados, vemos a real natureza de toda problemática causada pelo vírus, onde ele se espalha muito rapidamente e que pode gerar congestionamento ao sistema de saúde do país, já que não teremos leitos, nem materiais de tratamento disponíveis se continuassemos a vida se distanciamento.

Assim, fica aberto para realizar as mesmas análises nas granularidades menores, como Estado e Municípios e verificar em quais localidades tivemos melhores e piores evoluções.